In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 100     # number of variables
num_ineq = 100    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq)

## Heuristic - Round

In [7]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_100-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [06:23<00:00,  2.60it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -21.886359          5.596299      0.173362
std       0.462866          1.133289      0.011438
min     -23.293625          2.305694      0.157268
25%     -22.198093          4.785628      0.166839
50%     -21.885911          5.557468      0.170520
75%     -21.562072          6.319377      0.176169
max     -20.401462          9.460674      0.245447
Number of infeasible solution: 1000


## Heuristic - N1

In [9]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


In [10]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_n1_100-100.csv")

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [11:55:24<00:00, 42.92s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.587003e+18               0.0     42.896583
std    5.831904e+17               0.0     16.178696
min   -1.834646e+01               0.0      4.436790
25%    1.193007e+18               0.0     30.471017
50%    1.486973e+18               0.0     41.963484
75%    1.871542e+18               0.0     54.495735
max    4.563401e+18               0.0    102.884029
Number of infeasible solution: 0


## Learnable Rounding

In [11]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [12]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [13]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [14]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2041.92
Epoch 1, Validation Loss: -0.22
Epoch 2, Validation Loss: -6.79
Epoch 3, Validation Loss: -8.36
Epoch 4, Validation Loss: -8.58
Epoch 5, Validation Loss: -9.41
Epoch 6, Validation Loss: -9.91
Epoch 7, Validation Loss: -9.91
Epoch 8, Validation Loss: -9.42
Epoch 9, Validation Loss: -10.29
Epoch 10, Validation Loss: -10.47
Epoch 11, Validation Loss: -10.44
Epoch 12, Validation Loss: -10.69
Epoch 13, Validation Loss: -11.21
Epoch 14, Validation Loss: -11.78
Epoch 15, Validation Loss: -11.10
Epoch 16, Validation Loss: -11.84
Epoch 17, Validation Loss: -11.40
Epoch 18, Validation Loss: -11.63
Epoch 19, Validation Loss: -11.95
Epoch 20, Validation Loss: -11.88
Epoch 21, Validation Loss: -12.15
Epoch 22, Validation Loss: -12.32
Epoch 23, Validation Loss: -11.92
Epoch 24, Validation Loss: -12.42
Epoch 25, Validation Loss: -12.61
Epoch 26, Validation Loss: -12.52
Epoch 27, Validation Loss: -12.64
Epoch 28, Validation Loss: -12.54
Epoch 29, Validation Loss: -12.

In [15]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_100-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:44<00:00, 22.25it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -15.118945          0.008103      0.004625
std       0.198483          0.036200      0.001903
min     -15.686035          0.000000      0.001996
25%     -15.259828          0.000000      0.003369
50%     -15.119030          0.000000      0.004068
75%     -14.993435          0.000000      0.005229
max     -14.368282          0.313173      0.013949
Number of infeasible solution: 79


## Learnable Threshold

In [16]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [17]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [18]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [19]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1242.51
Epoch 1, Validation Loss: -1.68
Epoch 2, Validation Loss: -9.92
Epoch 3, Validation Loss: -10.86
Epoch 4, Validation Loss: -11.97
Epoch 5, Validation Loss: -12.31
Epoch 6, Validation Loss: -12.32
Epoch 7, Validation Loss: -12.46
Epoch 8, Validation Loss: -12.06
Epoch 9, Validation Loss: -12.10
Epoch 10, Validation Loss: -12.51
Epoch 11, Validation Loss: -12.51
Epoch 12, Validation Loss: -11.55
Epoch 13, Validation Loss: -12.47
Epoch 14, Validation Loss: -12.61
Epoch 15, Validation Loss: -12.42
Epoch 16, Validation Loss: -12.19
Epoch 17, Validation Loss: -12.26
Epoch 18, Validation Loss: -12.56
Epoch 19, Validation Loss: -12.98
Epoch 20, Validation Loss: -12.22
Epoch 21, Validation Loss: -12.79
Epoch 22, Validation Loss: -12.49
Epoch 23, Validation Loss: -12.84
Epoch 24, Validation Loss: -13.14
Epoch 25, Validation Loss: -13.17
Epoch 26, Validation Loss: -12.91
Epoch 27, Validation Loss: -12.34
Epoch 28, Validation Loss: -12.99
Epoch 29, Validation Loss

In [20]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_100-100.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:35<00:00, 28.43it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -14.272195          0.007440      0.004104
std       0.285247          0.037330      0.002224
min     -14.728096          0.000000      0.001994
25%     -14.466759          0.000000      0.002904
50%     -14.376406          0.000000      0.003018
75%     -14.134510          0.000000      0.004820
max     -12.271348          0.440128      0.013719
Number of infeasible solution: 66
